# LangSmith Demo 🦜🛠️

LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. What we'll cover...
1. **Tracing** - Monitor LLM application runs with detailed observability
2. **Playground & Prompts** - Optimize and collaborate on prompts
3. **Datasets & Evaluations** - Test applications systematically
4. **Annotation Queues** - Enable human feedback and collaboration
5. **Automations & Online Evaluations** - Production monitoring
6. **Dashboards & Alerts** - Visualize application performance

## RAG Application

![Simple RAG](./images/simple_rag.png)

#### Setup

In [ ]:
import os
import tempfile
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# os.environ["OPENAI_API_KEY"] = ""
# os.environ["LANGCHAIN_API_KEY"] = ""
# os.environ["LANGSMITH_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "langsmith-demo"

from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

nest_asyncio.apply()

# Configuration
MODEL_NAME = "gpt-4o-mini"
MODEL_PROVIDER = "openai"

# RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
# Use the following pieces of retrieved context to answer the latest question in the conversation. 
# If you don't know the answer, just say that you don't know. 
# Use three sentences maximum and keep the answer concise.
# """

# Note that we are pulling our prompt from LangChain's Hub
prompt = hub.pull("ls-demo-v1")

USER_AGENT environment variable not set, consider setting it to identify your requests.


#### Setup VectorDB retriever

In [ ]:
def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd, persist_path=persist_path, serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    print("Indexing LangSmith documentation...")
    ls_docs_sitemap_loader = SitemapLoader(
        web_path="https://docs.smith.langchain.com/sitemap.xml",
        continue_on_failure=True,
    )
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet",
    )
    vectorstore.persist()
    print(f"Indexed {len(doc_splits)} document chunks")
    return vectorstore.as_retriever(lambda_mult=0)

# Initialize retriever
retriever = get_vector_db_retriever()

#### Main RAG application

In [ ]:
from openai.types.chat import ChatCompletion, ChatCompletionMessageParam
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(
    run_type="llm",
    metadata={"ls_provider": MODEL_PROVIDER, "ls_model_name": MODEL_NAME}
)
def call_openai(messages: List[ChatCompletionMessageParam]) -> ChatCompletion:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    formatted_prompt = prompt.invoke({"context":formatted_docs, "question": question})
    messages = convert_prompt_to_openai_format(formatted_prompt)["messages"]
    return call_openai(messages)

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

### Tracing: Observability for LLM Applications

LangSmith provides comprehensive tracing based on the open-source OpenTelemetry standard.

In [ ]:
question = "What is LangSmith used for?"
answer = langsmith_rag(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

langsmith-demo-3


### Playground & Prompts: Optimize and Collaborate

The Playground enables prompt optimization and collaboration.

In [ ]:
# Refresh prompt from LangChain hub
prompt = hub.pull("ls-demo-v1")

question = "What is LangSmith used for?"
answer = langsmith_rag(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Let's add some more examples

In [ ]:
# Refresh prompt from LangChain hub
prompt = hub.pull("ls-demo-v1")

questions = [
    "How do I set up LangSmith tracing?",
    "What are the key benefits of using LangSmith?",
    "Can LangSmith work without LangChain?"
]

for q in questions:
    answer = langsmith_rag(q)
    print(f"Q: {q}")
    print(f"A: {answer}\n")

### Datasets & Evaluations: Systematic Testing

Datasets are collections of test data for evaluating your application.

In [ ]:
from langsmith import Client

example_inputs = [
("How do I set up tracing to LangSmith if I'm using LangChain?", "To set up tracing to LangSmith while using LangChain, you need to set the environment variable `LANGSMITH_TRACING` to 'true'. Additionally, you must set the `LANGSMITH_API_KEY` environment variable to your API key. By default, traces will be logged to a project named \"default.\""),
("How can I trace with the @traceable decorator?", "To trace with the @traceable decorator in Python, simply decorate any function you want to log traces for by adding `@traceable` above the function definition. Ensure that the LANGSMITH_TRACING environment variable is set to 'true' to enable tracing, and also set the LANGSMITH_API_KEY environment variable with your API key. By default, traces will be logged to a project named \"default,\" but you can configure it to log to a different project if needed."),
("How do I pass metadata in with @traceable?", "You can pass metadata with the @traceable decorator by specifying arbitrary key-value pairs as arguments. This allows you to associate additional information, such as the execution environment or user details, with your traces. For more detailed instructions, refer to the LangSmith documentation on adding metadata and tags."),
("What is LangSmith used for in three sentences?", "LangSmith is a platform designed for the development, monitoring, and testing of LLM applications. It enables users to collect and analyze unstructured data, debug issues, and create datasets for testing and evaluation. The tool supports various workflows throughout the application development lifecycle, enhancing the overall performance and reliability of LLM applications."),
("What testing capabilities does LangSmith have?", "LangSmith offers capabilities for creating datasets of inputs and reference outputs to run tests on LLM applications, supporting a test-driven approach. It allows for bulk uploads of test cases, on-the-fly creation, and exporting from application traces. Additionally, LangSmith facilitates custom evaluations to score test results, enhancing the testing process."),
("Does LangSmith support online evaluation?", "Yes, LangSmith supports online evaluation as a feature. It allows you to configure a sample of runs from production to be evaluated, providing feedback on those runs. You can use either custom code or an LLM as a judge for the evaluations."),
("Does LangSmith support offline evaluation?", "Yes, LangSmith supports offline evaluation through its evaluation how-to guides and features for managing datasets. Users can manage datasets for offline evaluations and run various types of evaluations, including unit testing and auto-evaluation. This allows for comprehensive testing and improvement of LLM applications."),
("Can LangSmith be used for finetuning and model training?", "Yes, LangSmith can be used for fine-tuning and model training. It allows you to capture run traces from your deployment, query and filter this data, and convert it into a format suitable for fine-tuning models. Additionally, you can create training datasets to keep track of the data used for model training."),
("Can LangSmith be used to evaluate agents?", "Yes, LangSmith can be used to evaluate agents. It provides various evaluation strategies, including assessing the agent's final response, evaluating individual steps, and analyzing the trajectory of tool calls. These methods help ensure the effectiveness of LLM applications."),
("How do I create user feedback with the LangSmith sdk?", "To create user feedback with the LangSmith SDK, you first need to run your application and obtain the `run_id`. Then, you can use the `create_feedback` method, providing the `run_id`, a feedback key, a score, and an optional comment. For example, in Python, it would look like this: `client.create_feedback(run_id, key=\"feedback-key\", score=1.0, comment=\"comment\")`."),
]

client = Client()
# TODO: Create dataset and fill in dataset id
dataset_id = ""

inputs = [{"question": input_prompt} for input_prompt, _ in example_inputs]
outputs = [{"output": output_answer} for _, output_answer in example_inputs]

client.create_examples(
  inputs=inputs,
  outputs=outputs,
  dataset_id=dataset_id,
)

### Custom Evaluators

LangSmith supports both *LLM-as-Judge* evaluators and custom *code* evaluators. Examples:
- Check if an answer is grounded in the provided documents
- Score the perceived helpfulness of an answer from 1-10
- Validate that an output contains valid Python
- For an email assistant, use regex to check that the correct email signature is used

#### LLM-as-Judge evaluator

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langsmith.evaluation import EvaluationResult
from pydantic import BaseModel, Field

class Similarity_Score(BaseModel):
    similarity_score: int = Field(description="Similarity score between 1 and 10, where 1 means unrelated and 10 means identical.")

def similarity(inputs: dict, outputs: dict, reference_outputs: dict) -> dict:
    prompt = """
    You are a similarity evaluator comparing the meaning of two response outputs to an input.
    The reference_output is the correct answer to the input. The output is what we want to grade to make sure it is similar to the reference_output.
    Your task is to assign a score 1-10 based on the following rubric:

    <Rubric>
        When scoring, you should reward:
        - When the output and the reference_output are semantically similar
        - When the output and the reference_output are logically similar

        When scoring, you should penalize:
        - If the output contradicts the reference_output
        - If the output is logically inconsistent from the reference_output
        - If the output is off topic from the input and reference_output
    </Rubric>

    <Instructions>
        - Carefully read the input, output, and reference_output
        - Use the reference_output to determine if output contains errors or logical inconsistencies 
    </Instructions>

    <Reminder>
        The reference_output is the correct answer to the input and we want to evaluate the output.
    </Reminder>

    <input>
        {}
    </input>

    <output>
        {}
    </output>

    Use the reference outputs below to help you evaluate the correctness of the response:
    <reference_outputs>
        {}
    </reference_outputs>
    """.format(inputs["question"], outputs["output"], reference_outputs["output"])
    structured_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0).with_structured_output(Similarity_Score)
    generation = structured_llm.invoke([HumanMessage(content=prompt)])
    return {"key": "similarity", "score": generation.similarity_score}

#### Code Evaluator

In [ ]:
def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.25 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

#### Run Evaluators

In [ ]:
from langsmith import evaluate, Client

client = Client()
# TODO add dataset name
dataset_name = ""

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate( 
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough, similarity],
    experiment_prefix="ls-demo-1"
)

### Annotation Queues: Human Feedback & Collaboration

Annotation queues enable developers and subject matter experts to provide structured feedback.

In [ ]:
# Create examples to add to annotation queue
annotation_questions = [
    "What are the pricing tiers for LangSmith?",
    "How does LangSmith compare to other LLM monitoring tools?",
    "Can I use LangSmith for fine-tuning models?",
    "What are hosting options for LangSmith?"
]

for question in annotation_questions:
    answer = langsmith_rag(question)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print("-" * 80)

### Automations & Online Evaluations: Production Monitoring

Automations run on live production interactions to provide continuous monitoring. Examples:
- Sample all runs with low frequency and add to annotation queue for human spot checking
- Add traces with negative feedback to annotation queue for diagnosis
- Add all traces with positive feedback to a golden dataset
- For all traces with an error, alert PagerDuty or create Jira ticket

In [ ]:
import random
import time

production_questions = [
    "How do I get started with LangSmith?",
    "What's the difference between tracing and evaluation?",
    "How can I monitor my LLM costs?",
    "What integrations does LangSmith support?",
    "How do I export my data from LangSmith?",
    "Can I use custom models with LangSmith?"
]

for i in range(3):
    question = random.choice(production_questions)
    
    answer = langsmith_rag(question)
        
    print(f"Session {i+1}: {question}")
    print(f"Answer: {answer}\n")

### Dashboards: Visualize Performance

LangSmith provides both pre-built and custom dashboards for monitoring:

**Pre-built Dashboards**:
- Request volume and latency
- Cost tracking by model
- Error rates and types
- Token usage patterns

**Custom Dashboards**:
- Multi-project views
- Custom metrics and filters
- Business-specific KPIs

**Alerts**
- Based on errors, feedback, or latency
- PagerDuty or Webhook integrations